# <center><font color=#76B900 size="+3"><b>**NVIDIA cuOpt for CVRPTW**</b></font></center>
---

**Learning Objectives:**
- Introducing multiple cosntrints to model more complex problems
- Using real-world data and running data preprocessing
- Integrating third-party tools for traffic data
  
In this notebook, we will use cuOpt to solve a real world application of a Capacitated Vehicle Routing Problem with Time Windows (CVRPTW). This use case is also called Last Mile Delivery (LMD).

Suppose you are working as an Optimization Scientist at a grocery chain throughout New York City. There are 97 stores and 3 distribution centers. Every day, stores place an order for food that must be delivered the next day to ensure they are fully stocked. Given input data about stores' orders, distribution centers, and the available fleet of vehicles, it is your job to calculate the route for each vehicle such that all orders are fulfilled while minimizing vehicles' travel time and cost. For a problem space of 100 locations there are about 100! possible routes. You can do the math yourself- but that's a lot. Lucky for you, you have access to the cuOpt solver. All you need to do is read the input data and preprocess the data. Once all the data is ready, you just need to save it to one Python dictionary and send it to cuOpt, which does all the hard computation for you.

Let's walk through these steps. 

For the Last Mile Delivery (LMD) use case, we need 3 datasets with the following features:

- Depots
    - Name
    - Location
    - Start and end time (operation hours)
- Orders
    - Store Name
    - Location
    - Start and end time (store hours)
    - Demand
    - Service time
    - Loyalty Member
    - Delivery Requirement
- Vehicles
    - Name/ID Number
    - Assigned depot
    - Start and end time (vehicle/driver shift hours)
    - Break time
    - Capacity
    - Max time
    
You may have additional features depending on the problem at hand.


<br>

## Read input data from CSV files

Here, we are using real world addresses instead of points in the Euclidean space. Location data needs to be in the form of coordinate points (longitude and latitude values). Our data already has coordinate points included. If you're using new data and need to do so yourself, you will need to use a third party tool.

In this workflow, we are using locations from the following [Kaggle dataset](https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata). This is a dataset of Airbnbs in New York City. Our problem space has 100 locations total which includes 3 depots and 97 orders. The coordinate points are taken from the dataset and the rest of the features are synthetic data. We have 15 vehicles available.

In [2]:
DATA_PATH = "data/"

orders_df = pd.read_csv(DATA_PATH+"orders_lmd.csv")
depots_df = pd.read_csv(DATA_PATH+"depots_lmd.csv")
vehicles_df = pd.read_csv(DATA_PATH+"vehicles_lmd.csv")

In [3]:
n_depots = len(depots_df.index)
n_orders = len(orders_df.index)
n_vehicles = len(vehicles_df.index)

n_loc_total = n_orders + n_depots

In [4]:
locations_df = (pd.concat([depots_df[["Name","Longitude","Latitude"]], orders_df[["Name","Longitude","Latitude"]]], ignore_index=True)).reset_index()

<br>

## Create Cost Matrix

The **cost matrix** models the cost between each pair of locations.  It is used by cuOpt to compute the cost of traveling from any location to any other. The cost matrix needs to be a square matrix of dimension equal to the total number of locations which inlcludes both depots and orders. In this Vehicle Routing Problem, our cost metric is travel time. This is cost we want to minimize. 

To build a a cost matrix of live traffic data, we need to use a third party map data provider. In this workflow, the cost matrix will calculate the travel time in minutes between each two pairs of locations which we build using [OSRM](https://project-osrm.org/).


In practical applications, you can integrate this to a third-party map data provider like Esri or Google Maps to get live traffic data and run dynamic/real-time re-routing using cuOpt.

In [13]:
 def build_travel_time_matrix(df):
    latitude = df.Latitude.to_numpy()
    longitude = df.Longitude.to_numpy()
    
    locations=""
    n_orders = len(df)
    for i in range(n_orders):
        locations = locations + "{},{};".format(longitude[i], latitude[i])
    r = requests.get("http://router.project-osrm.org/table/v1/car/"+ locations[:-1])
    routes = json.loads(r.content)
    
    # OSRM returns duration in seconds. Here we are converting to minutes
    for i in routes['durations']:
        i[:] = [x / 60 for x in i]
    
    coords_index = { i: (latitude[i], longitude[i]) for i in range(df.shape[0])}
    time_matrix = pd.DataFrame(routes['durations'])
    
    return time_matrix

In [14]:
cost_matrix_df = build_travel_time_matrix(locations_df)

<br>

## Preprocess Fleet and Task Data

Here, we take our raw input data and convert it to the format needed for cuOpt.

### Set Fleet Data

Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

- vehicle_locations is a list of the start and end location of the vehicles. Each vehicle is assigned to a depot from which it departs in the morning and returns to at night. For example, a vehicle that starts and ends in depot 1 which is the location at index 0 would have the vehicle location of [0,0]. 

- capacities is a list of how much goods each vehicle can carry in weight. Here we have two different types of vehicles: trucks and EV vans. A truck can carry up to 20,000 pounds and an EV van can carry up to 8,000 pounds. This is essential when assigning orders to vehicles because one vehicle can only carry so many orders at once. 

- vehicle_time_windows is a list of the integer representation of the operating time of each vehicle. Equivalently, the shift of each vehicle driver. We convert the UTC timestamp to epoch time (integer representation in minutes).

- vehicle_break_time_windows is a list of the integer representation of break time of each vehicle within its operating time. For a driver working an 8 hour shift, this break in the middle of the day represents their lunch break. These time windows are when their lunch break may occur.
  
- vehicle_break_durations is the length of the break. Here, we set the duration to be 30 minutes for all vehicles. 

- vehicle_max_time is a list of the maximum time a vehicle can operate. Even if a driver is available for a long period of time, this constraint enforces a maximum length for a driver's shift. This is also given in minutes. A driver's time window represents total availability which may be longer than a standard shift length. If a driver says they are available to work from 9am to 9pm, we still want to limit their shift to be shorter. A truck driver can drive up to 7 hours, and an EV driver can drive up to 4 hours. 

In [6]:
depot_names_to_indices_dict = {locations_df["Name"].values.tolist()[i]: i for i in range(n_depots)}
vehicle_locations = vehicles_df[["assigned_depot","assigned_depot"]].replace(depot_names_to_indices_dict).values.tolist()

In [7]:
capacities = [[int(a) for a in vehicles_df['vehicle_capacity'].tolist()]]

In [8]:
vehicle_time_windows = pd.concat((vehicles_df['vehicle_start'].apply(get_minutes_from_datetime).to_frame(), vehicles_df['vehicle_end'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()

In [9]:
vehicle_break_time_windows = [pd.concat((vehicles_df['break_start'].apply(get_minutes_from_datetime).to_frame(), vehicles_df['break_end'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()]

In [10]:
vehicle_break_durations = [[30] * n_vehicles]

In [11]:
vehicles_max_time = vehicles_df['max_time'].tolist()

### Set Task Data


Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

- task_locations is the list of stores that have placed an order. This list is simply the index of each location. 

- task_time_windows is the list of integer representation of opening hours for each store. We convert the UTC timestamp to epoch time (integer representation in minutes).

- service_times is the list of the length of time for orders to be dropped off once the vehicle reaches the location. Here, these values are between 15 and 30 minutes.

- demand is the list of weight demand for each order. Here, these values are between 40 and 200 pounds. 

- vehicle_match_list allows us to ensure that some orders are assigned to specific vehicles. In this use case, some of the orders are frozen and can be delivered in trucks and not EV vans. Here we can indicate that the frozen orders are assigned specifically to vehicles that are trucks.  

In [15]:
task_locations = locations_df.index.tolist()[n_depots:]

In [16]:
demands = [orders_df['Demand'].values.tolist()]

In [17]:
service_times = orders_df['ServiceTime'].values.tolist()

In [18]:
task_time_windows = pd.concat((orders_df['order_start_time'].apply(get_minutes_from_datetime).to_frame(), orders_df['order_end_time'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()

In [19]:
trucks_ids = vehicles_df['vehicle_type'][vehicles_df['vehicle_type']=="Truck"].index.values.tolist()

In [20]:
vehicle_match_list = []
for i in orders_df['is_frozen'][orders_df['is_frozen']==1].index.values.tolist():
    vehicle_match_list.append({"order_id": i, "vehicle_ids": trucks_ids})

<br>

## Save data in a dictionary

Here, we take all the data we have prepared so far and save it to one dictionary. This includes the cost matrices, task data, fleet data, and solver config. This is all the data that cuOpt needs to solve our LMD problem. 

In [34]:
cuopt_problem_data = {
    "cost_matrix_data": {
        "cost_matrix": {
            "0": cost_matrix_df.to_numpy().tolist()
        }
    },
    "travel_time_matrix_data": {
        "cost_matrix": {
            "0": cost_matrix_df.to_numpy().tolist()
        }
    },
    "task_data": {
        "task_locations": task_locations,
        "demand": demands,
        "task_time_windows": task_time_windows,
        "service_times":service_times,
        "order_vehicle_match": vehicle_match_list,
    },
    "fleet_data": {
        "vehicle_locations": vehicle_locations,
        "capacities": capacities,
        "vehicle_time_windows": vehicle_time_windows,
        "vehicle_break_time_windows": vehicle_break_time_windows,
        "vehicle_break_durations": vehicle_break_durations,
        "vehicle_max_times": vehicles_max_time,
    },
    "solver_config": {
        "time_limit": 0.1,
    }
}

In [35]:
import json
# Convert and write JSON object to file
with open("cuopt_data_lmd.json", "w") as outfile: 
    json.dump(cuopt_problem_data, outfile)